In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('/FileStore/tables/Ecommerce_Customers.csv', header=True, inferSchema=True)

In [5]:
data.show(4)

+--------------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
 Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
mstephenson@ferna...|835 Frank TunnelW...| Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616| 4.0826206329529615| 587.9510539684005|
 hduke@hotmail.com|4547 Archer Commo...| DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744| 2.66403418213262| 392.2049334443264|
 pallen@yahoo.com|24645 Valerie Uni...| Bisque|33.000914755642675|11.330278057777512|37.110597442120856| 4.104543202376424| 487.54750486747207|
riverarebecca@gma...|1414 David Throug...|SaddleBrown| 34.30555662975554|13.717513665142507| 36.72128267790313| 3.120178782748092| 581.8523440352177|
+--------------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
only showing top 4 rows

In [6]:
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [7]:
for item in data.head(2)[1]:
  print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

Out[ 13 ]: 
['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
# create an assembler object and tell is what to expect
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership',
                                       'Yearly Amount Spent'],
                             outputCol='features')

In [11]:
type(assembler)

Out[ 54 ]: pyspark.ml.feature.VectorAssembler

In [12]:
output = assembler.transform(data)

In [13]:
type(output)

Out[ 55 ]: pyspark.sql.dataframe.DataFrame

In [14]:
output.select('features').show(4)

+--------------------+
 features|
+--------------------+
[34.4972677251122...|
[31.9262720263601...|
[33.0009147556426...|
[34.3055566297555...|
+--------------------+
only showing top 4 rows

In [15]:
# note that features is now a DenseVector of the values input when creating the assembler objcet
output.head(1)

Out[ 23 ]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [16]:
final_data = output.select('features','Yearly Amount Spent')

In [17]:
final_data.show(4)

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
+--------------------+-------------------+
only showing top 4 rows

In [18]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [19]:
train_data.describe().show(4)

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 365|
 mean| 499.23611553194024|
 stddev| 79.18579218078546|
 min| 256.67058229005585|
+-------+-------------------+
only showing top 4 rows

In [20]:
test_data.describe().show(4)

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 135|
 mean| 499.52471822323935|
 stddev| 79.95784329870017|
 min| 298.76200786180766|
+-------+-------------------+
only showing top 4 rows

In [21]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [22]:
lr_model = lr.fit(train_data)

In [23]:
test_results = lr_model.evaluate(test_data)

In [24]:
test_results.residuals.show(10)

+--------------------+
 residuals|
+--------------------+
 0.0|
5.684341886080801...|
 0.0|
 0.0|
 0.0|
 0.0|
5.684341886080801...|
 0.0|
 0.0|
5.684341886080801...|
+--------------------+
only showing top 10 rows

In [25]:
test_results.rootMeanSquaredError

Out[ 40 ]: 2.4461512732155778e-14

In [26]:
test_results.r2

Out[ 41 ]: 1.0

In [27]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [28]:
lr_model

Out[ 43 ]: LinearRegression_5cca991ff535

In [29]:
lr_model.coefficients

Out[ 44 ]: DenseVector([-0.0, -0.0, 0.0, -0.0, 1.0])

In [30]:
lr_model.intercept

Out[ 45 ]: 0.0

In [31]:
unlabeled_data = test_data.select('features')

In [32]:
unlabeled_data.show(4)

+--------------------+
 features|
+--------------------+
[29.5324289670579...|
[30.8364326747734...|
[31.0472221394875...|
[31.0662181616375...|
+--------------------+
only showing top 4 rows

In [33]:
predictions = lr_model.transform(unlabeled_data)

In [34]:
predictions.show(4)

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...| 408.6403510726275|
[30.8364326747734...|467.50190042698955|
[31.0472221394875...| 392.4973991890214|
[31.0662181616375...|448.93329320767435|
+--------------------+------------------+
only showing top 4 rows